# 1. 필요한 패키지 불러오기

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from konlpy.tag import Komoran
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential
import os
import keras
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.initializers import glorot_uniform
import pickle


import warnings
warnings.filterwarnings(action='ignore')

Using TensorFlow backend.


# 2. get_prediction 함수

1. 이 함수는 테스트에 해당하는 csv 파일 경로를 매개변수로 가집니다.
2. 이 함수는 미리 만들어진 모델을 불러와 예측을 수행합니다. 
3. 이 함수는 예측치를 불러온 csv 파일에 넣습니다. 이 때 예측치에 해당하는 column name은 smishing 입니다.
4. 이 함수는 데이터프레임을 리턴합니다.

In [25]:
def get_prediction(test_file_path):
    '''
    Arg.: String
    Return: Pandas DataFrame
    '''
    
    '''1. 테스트 파일 불러오기'''
    
    test = pd.read_csv(test_file_path)
    
    
    '''2. 모델 불러오기'''
    
    os.chdir('../1_Model')
    
    # 토큰화 된 벡터에 배정 될 명사 단어 집합 불러오기
    with open('noun_sets.pickle', 'rb') as f:
         noun_sets = pickle.load(f)
    
    with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        
    # model1, model2 불러오기
            model1 = load_model('model1.h5')
            model2 = load_model('model2.h5')
        
    '''3. 테스트 데이터 전처리'''
    
    temp = test.copy()

    # Komoran을 통해 형태소 분석 후, 명사만 추출하여 test_noun_result 리스트에 저장함
    komoran = Komoran()
    docs = temp.text
    test_noun_result = list(map(komoran.nouns, docs))
    temp['result'] = test_noun_result   
    
    # 명사 단어 집합에 없는 단어 삭제하기
    results = []

    def check_nouns(x):
        if x in noun_sets:
            return x
        else:
            return None

    for result in temp['result']:
        result = list(filter(check_nouns, result))
        results.append(result)
    
    temp['result'] = results
    
    X_data = temp['result']
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_data) #X_data의 각 행에 토큰화 수행
    
    # 각 단어를 저장된 명사 집합의 번호와 일치하게 숫자로 변환
    tot = []
    for result in X_data:
        now = []
        for noun in result:
            now.append(noun_sets[noun])
        tot.append(now)
        
    temp['result'] = tot

    X_data = temp['result']
    
    max_len = max(len(l) for l in X_data) #최대 길이로 맞춤
    data = pad_sequences(X_data, maxlen=max_len)
        
        
    
    '''4. 예측 진행하기'''
    
    X_test = data.copy()
    
    # model1의 predict 값을 temp의 smishing1 컬럼에 저장
    temp['smishing1'] = model1.predict(X_test)
    
    # model2의 predict 값을 temp의 smishing2 컬럼에 저장
    temp['smishing2'] = model2.predict(X_test)
    
    # model1과 model2의 6:4 weighted mean 앙상블로 최종 예측값 출력하여 temp의 smishing 칼럼에 저장
    temp['smishing'] = (temp['smishing1']*6 + temp['smishing2']*4)/10
    
    
    '''5. 예측치 데이터프레임에 합치기'''
    
    test['smishing'] = temp['smishing']
    
    return test

In [26]:
get_prediction('../0_Data/public_test.csv').head()

id year_month                                               text  \
0  340000    2019-01  XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...   
1  340001    2019-01   긴급 안내  XXX은행 가락동 지점  - 헬리오XXX 기본XXX    대출이자를 ...   
2  340002    2019-01  XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...   
3  340003    2019-01  XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...   
4  340004    2019-01  XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다.  고...   

       smishing  
0  4.172325e-08  
1  1.321435e-05  
2  5.364418e-08  
3  9.536743e-08  
4  5.364418e-08

### a) 인풋데이터 예시

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>id</th>
      <th>year_month</th>
      <th>text</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>340000</td>
      <td>2019-01</td>
      <td>XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>340001</td>
      <td>2019-01</td>
      <td>긴급 안내  XXX은행 가락동 지점  - 헬리오XXX 기본XXX    대출이자를 ...</td>
    </tr>
    <tr>
      <th>2</th>
      <td>340002</td>
      <td>2019-01</td>
      <td>XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...</td>
    </tr>
    <tr>
      <th>3</th>
      <td>340003</td>
      <td>2019-01</td>
      <td>XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...</td>
    </tr>
    <tr>
      <th>4</th>
      <td>340004</td>
      <td>2019-01</td>
      <td>XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다.  고...</td>
    </tr>
    <tr>
      <th>5</th>
      <td>340005</td>
      <td>2019-01</td>
      <td>기기분좋게해해피하게년년중내내1월의바램과 소원 쭈욱겨울의 꽃 1월행복으로 마감되는눈을...</td>
    </tr>
    <tr>
      <th>6</th>
      <td>340006</td>
      <td>2019-01</td>
      <td>XXX 고객님 안녕하세요.XXX은행 XXX지점 VIP팀장 XXX입니다.제가 이번에 ...</td>
    </tr>
    <tr>
      <th>7</th>
      <td>340007</td>
      <td>2019-01</td>
      <td>(광고)XXX 고객님XXX외화 환전 서비스 안내XXX은행 고현종합금융센터입니다.외화...</td>
    </tr>
    <tr>
      <th>8</th>
      <td>340008</td>
      <td>2019-01</td>
      <td>XXX 고객님우리가 쉽게 지나치는세잎클로버의 꽃 말이행복이라는 사실 알고계셨나요.늘...</td>
    </tr>
    <tr>
      <th>9</th>
      <td>340009</td>
      <td>2019-01</td>
      <td>- 법인계좌 압류등록 안내    2018.11.23  3115790원 한울상사   ...</td>
    </tr>
  </tbody>
</table>

### b) 아웃풋데이터 예시

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>id</th>
      <th>year_month</th>
      <th>text</th>
      <th>smishing</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>340000</td>
      <td>2019-01</td>
      <td>XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...</td>
      <td>0.200</td>
    </tr>
    <tr>
      <th>1</th>
      <td>340001</td>
      <td>2019-01</td>
      <td>긴급 안내  XXX은행 가락동 지점  - 헬리오XXX 기본XXX    대출이자를 ...</td>
      <td>0.413</td>
    </tr>
    <tr>
      <th>2</th>
      <td>340002</td>
      <td>2019-01</td>
      <td>XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...</td>
      <td>0.560</td>
    </tr>
    <tr>
      <th>3</th>
      <td>340003</td>
      <td>2019-01</td>
      <td>XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...</td>
      <td>0.990</td>
    </tr>
    <tr>
      <th>4</th>
      <td>340004</td>
      <td>2019-01</td>
      <td>XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다.  고...</td>
      <td>0.010</td>
    </tr>
    <tr>
      <th>5</th>
      <td>340005</td>
      <td>2019-01</td>
      <td>기기분좋게해해피하게년년중내내1월의바램과 소원 쭈욱겨울의 꽃 1월행복으로 마감되는눈을...</td>
      <td>0.430</td>
    </tr>
    <tr>
      <th>6</th>
      <td>340006</td>
      <td>2019-01</td>
      <td>XXX 고객님 안녕하세요.XXX은행 XXX지점 VIP팀장 XXX입니다.제가 이번에 ...</td>
      <td>0.120</td>
    </tr>
    <tr>
      <th>7</th>
      <td>340007</td>
      <td>2019-01</td>
      <td>(광고)XXX 고객님XXX외화 환전 서비스 안내XXX은행 고현종합금융센터입니다.외화...</td>
      <td>0.320</td>
    </tr>
    <tr>
      <th>8</th>
      <td>340008</td>
      <td>2019-01</td>
      <td>XXX 고객님우리가 쉽게 지나치는세잎클로버의 꽃 말이행복이라는 사실 알고계셨나요.늘...</td>
      <td>0.110</td>
    </tr>
    <tr>
      <th>9</th>
      <td>340009</td>
      <td>2019-01</td>
      <td>- 법인계좌 압류등록 안내    2018.11.23  3115790원 한울상사   ...</td>
      <td>0.200</td>
    </tr>
  </tbody>
</table>